In [1]:
import numpy as np
import pandas as pd

In [2]:
from typing import List, Dict

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
songs = pd.read_csv('../data/songdata.csv')

In [8]:
songs.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [9]:
songs

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
...,...,...,...,...
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \nLet the angels fly l...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \nMore power \nPower to...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \nis something i'll believe \nf...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \nam i frightened \nwhere can ...


In [10]:
songs = songs.sample(n=5000).drop('link', axis=1).reset_index(drop=True)


In [11]:
songs['text'] = songs['text'].str.replace(r'\n', '')

/var/folders/5b/_80n_r3s42d_yc94yjfdh__c0000gn/T/ipykernel_16918/269676193.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  songs['text'] = songs['text'].str.replace(r'\n', '')


In [12]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')

In [13]:
lyrics_matrix = tfidf.fit_transform(songs['text'])

In [14]:
cosine_similarities = cosine_similarity(lyrics_matrix) 

In [15]:

similarities = {}

In [16]:
for i in range(len(cosine_similarities)):
    # Now we'll sort each element in cosine_similarities and get the indexes of the songs. 
    similar_indices = cosine_similarities[i].argsort()[:-50:-1] 
    # After that, we'll store in similarities each name of the 50 most similar songs.
    # Except the first one that is the same song.
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], songs['artist'][x]) for x in similar_indices][1:]

In [17]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)
        
        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score") 
            print("--------------------")
        
    def recommend(self, recommendation):
        # Get song to find recommendations for
        song = recommendation['song']
        # Get number of songs to recommend
        number_songs = recommendation['number_songs']
        # Get the number of songs most similars from matrix similarities
        recom_song = self.matrix_similar[song][:number_songs]
        # print each item
        self._print_message(song=song, recom_song=recom_song)

In [18]:
recommedations = ContentBasedRecommender(similarities)

In [19]:
recommendation = {
    "song": songs['song'].iloc[10],
    "number_songs": 4 
}

In [20]:
recommedations.recommend(recommendation)

The 4 recommended songs for A Little Time are:
Number 1:
Put A Little Love In Your Heart by Aiza Seguerra with 0.412 similarity score
--------------------
Number 2:
A Little A This N That by Arlo Guthrie with 0.404 similarity score
--------------------
Number 3:
Little Darling by Thin Lizzy with 0.376 similarity score
--------------------
Number 4:
Have A Little Faith In Me by Kenny Rogers with 0.366 similarity score
--------------------


In [21]:
recommendation2 = {
    "song": songs['song'].iloc[120],
    "number_songs": 4 
}

In [22]:
recommedations.recommend(recommendation2)

The 4 recommended songs for Feel Like Makin' Love are:
Number 1:
It's The Falling In Love by Michael Jackson with 0.455 similarity score
--------------------
Number 2:
Just Like A Pill by P!nk with 0.412 similarity score
--------------------
Number 3:
Makin Love by Fabolous with 0.407 similarity score
--------------------
Number 4:
You Make Me Feel by Kyla with 0.388 similarity score
--------------------
